In [2]:
! pip3 install onnxoptimizer==0.2.6
! pip list | grep onnx

Defaulting to user installation because normal site-packages is not writeable
onnx                          1.11.0
onnxoptimizer                 0.2.6
onnxruntime                   1.11.1
qonnx                         0.0.post1.dev134+g398a0ec /workspace/qonnx/src


In [11]:
from qonnx.core.modelwrapper import ModelWrapper
from qonnx.transformation.infer_shapes import InferShapes
from qonnx.transformation.general import GiveReadableTensorNames, GiveUniqueNodeNames
from qonnx.transformation.infer_data_layouts import InferDataLayouts
from qonnx.transformation.infer_datatypes import InferDataTypes
from qonnx.transformation.merge_onnx_models import MergeONNXModels
from qonnx.transformation.batchnorm_to_affine import BatchNormToAffine
from onnx.helper import make_graph, make_model, set_model_props
from onnx import helper, TensorProto
import qonnx.core.data_layout as DataLayout
from qonnx.core.datatype import DataType
import numpy as np
import onnx

def get_init(model, tensor_name):
    for init in model.graph.initializer:
        if tensor_name == init.name:
            init_val = init.raw_data
            model.graph.initializer.remove(init)
            break
    for inp in model.graph.input:
        if tensor_name in inp.name:
            model.graph.input.remove(inp)
    
    return np.frombuffer(init_val, dtype = np.int64)

def add_attr(model, op_type):
    new_node = []
    for ind,n in enumerate(model.graph.node):
        new_attr = {}
        if op_type in n.name:        
            if len(n.input) > 1:
                new_attr["starts"] = get_init(model,n.input[1])
                new_attr["ends"] = get_init(model,n.input[2])
                new_attr["axes"] = get_init(model,n.input[3])
            if len(n.input[1:]) >3:
                new_attr["steps"] = get_init(model,n.input[4])
            if not new_attr == {}:
                new_node = make_node(model,new_attr,op_type)
                model.graph.node.remove(n)
                model.graph.node.append(new_node)
    
    return model

def make_node(model,slice_attr,op_type):
    for n in model.graph.node:
        if op_type in n.name:
            node = helper.make_node(
                op_type,
                name = n.name,
                inputs=[n.input[0]],
                outputs=[n.output[0]],
                **slice_attr
            )
    
            return node
    
def new_model(model):
    """
    Overwrites the main opset in an ONNX file.
    Does not change any node definition.
    :param model: ONNX model
    :param new_opset: new opset
    :return: ONNX model
    """    
    
    model_config = {}
    model_config["opset_imports"] = [onnx.helper.make_operatorsetid("",9)]
    
    graph = make_graph(
        model.graph.node, model.graph.name, model.graph.input,
        model.graph.output, model.graph.initializer)
    onnx_model = make_model(graph, functions=model.functions,**model_config)
    onnx_model.ir_version = 4#model.ir_version
    onnx_model.producer_name = model.producer_name
    onnx_model.producer_version = model.producer_version
    onnx_model.domain = model.domain
    onnx_model.model_version = model.model_version
    onnx_model.doc_string = model.doc_string
    if len(model.metadata_props) > 0:  # pragma: no cover
        values = {p.key: p.value for p in model.metadata_props}
        set_model_props(onnx_model, values)

    return ModelWrapper(onnx_model)

def cnt_ops(model):
    nodes = []
    for n in model.graph.node:
        nodes.append(n.op_type)
    ops,cnts = np.unique(np.array(nodes),return_counts=True)
    total = {}
    for ind,k in enumerate(ops):
        total[str(k)] = cnts[ind]

    return total

In [12]:
def pre_process(model):
    
    global_in = model.graph.input[0]
    idt = model.get_tensor_datatype(global_in.name)

    node = helper.make_node(
        "Div",
        inputs = [global_in.name,"b"],
        outputs = ["y"],
        name = "Div_node"
    )
    model.graph.node.append(node)
    model.graph.node[0].input[0] = "y"
    model.set_initializer('b',np.array(255,dtype = np.float32))
#     model.set_tensor_datatype("y",DataType[])
    model.set_tensor_datatype(model.graph.input[0].name,DataType["UINT8"])
    model.set_tensor_layout(model.graph.input[0].name,DataLayout.NCHW)    
    
    return model

def update_bn(model):
    graph = model.graph
    for n in graph.node:
        if n.op_type == "BatchNormalization":
            new_bn_node = helper.make_node(
                "BatchNormalization",
                inputs = n.input,
                outputs = n.output,
                name = n.name
            ) 
            if len(n.attribute) > 0:
                for na in n.attribute:
                    new_bn_node.attribute.append(na)
            graph.node.remove(n)
            graph.node.append(new_bn_node)
    return model

In [13]:
model = ModelWrapper('tinyyolo-20210831.onnx')

model = add_attr(model, "Slice")
model = new_model(model.model)
# model = pre_process(model)
# model = update_bn(model)

model.save("tinyyolo-20210831_updated.onnx")
# model = model.transform(InferShapes())

print("Saved!")
# model.get_nodes_by_op_type("Slice")

Saved!


In [101]:
model = ModelWrapper("tinyyolo-20210831.onnx")
model.transform(BatchNormToAffine())

/workspace/qonnx/src/qonnx/core/modelwrapper.py:93: UserWarning: Some old-style domain attributes were automatically converted to new-style,
                i.e. domain=finn to domain=qonnx.custom_op.<general|fpgadataflow|...>
  warnings.warn(


TypeError: object of type 'NoneType' has no len()

In [102]:

import finn.builder.build_dataflow as build
from qonnx.core.modelwrapper import ModelWrapper
import finn.builder.build_dataflow_config as build_cfg
from custom_steps import (
    custom_step_tinyyolo_preprocess,
    custom_step_tinyyolo_streamline,
    custom_step_tinyyolo_lower,
    custom_step_tinyyolo_convert_to_hls,
    custom_step_partition
)

model_name = "tinyyolo-20210831"
# model_name = "tinyyolo-20210831_updated"
model_filename = "%s.onnx" % model_name


custom_steps = [
    "step_tidy_up",
    custom_step_tinyyolo_preprocess,
    custom_step_tinyyolo_streamline,
    custom_step_tinyyolo_lower,
    custom_step_tinyyolo_convert_to_hls,
    custom_step_partition,
    # "step_create_dataflow_partition",
    "step_target_fps_parallelization",
    "step_apply_folding_config",
    "step_generate_estimate_reports",
    "step_hls_codegen",
    "step_hls_ipgen",
    "step_set_fifo_depths",
    "step_create_stitched_ip",
    "step_measure_rtlsim_performance",
    "step_synthesize_bitfile",
    "step_make_pynq_driver",
    "step_deployment_package"
]

cfg = build_cfg.DataflowBuildConfig(
#     steps = custom_steps,
    output_dir="build-"+model_name, 
    synth_clk_period_ns = 10.0, 
    auto_fifo_depths = False,
    folding_config_file="tinyyolo-config-v0.2.json",
    board = "KV260_SOM",
    stitched_ip_gen_dcp=True,
    shell_flow_type=build_cfg.ShellFlowType.VIVADO_ZYNQ,
    generate_outputs = [
        build_cfg.DataflowOutputType.ESTIMATE_REPORTS,
        build_cfg.DataflowOutputType.STITCHED_IP,
        build_cfg.DataflowOutputType.RTLSIM_PERFORMANCE,
        build_cfg.DataflowOutputType.BITFILE,
        build_cfg.DataflowOutputType.PYNQ_DRIVER,
        build_cfg.DataflowOutputType.DEPLOYMENT_PACKAGE,
    ],
    # verification options
    # verify_steps = [
    #     build_cfg.VerificationStepType.STITCHED_IP_RTLSIM,
    # ],
    # verify_input_npy="sample_io/test_image_uint8_nhwc.npy",
    # verify_expected_output_npy="sample_io/test_pred.npy",
    # verify_save_full_context=True,
    # verify_save_rtlsim_waveforms=True,
)

build.build_dataflow_cfg(model_filename, cfg)



Building dataflow accelerator from tinyyolo-20210831.onnx
Intermediate outputs will be generated in /workspace/results
Final outputs will be generated in build-tinyyolo-20210831
Build log is at build-tinyyolo-20210831/build_dataflow.log
Running step: step_qonnx_to_finn [1/17]
Running step: step_tidy_up [2/17]


Traceback (most recent call last):
  File "/workspace/finn/src/finn/builder/build_dataflow.py", line 166, in build_dataflow_cfg
    model = transform_step(model, cfg)
  File "/workspace/finn/src/finn/builder/build_dataflow_steps.py", line 243, in step_tidy_up
    model = model.transform(FoldConstants())
  File "/workspace/qonnx/src/qonnx/core/modelwrapper.py", line 140, in transform
    (transformed_model, model_was_changed) = transformation.apply(transformed_model)
  File "/workspace/qonnx/src/qonnx/transformation/fold_constants.py", line 59, in apply
    oxe.execute_node(n, execution_context, graph)
  File "/workspace/qonnx/src/qonnx/core/onnx_exec.py", line 73, in execute_node
    sess = rt.InferenceSession(node_model.SerializeToString())
  File "/home/pgeel/.local/lib/python3.8/site-packages/onnxruntime/capi/onnxruntime_inference_collection.py", line 335, in __init__
    self._create_inference_session(providers, provider_options, disabled_optimizers)
  File "/home/pgeel/.local/lib/

> /home/pgeel/.local/lib/python3.8/site-packages/onnxruntime/capi/onnxruntime_inference_collection.py(372)_create_inference_session()
    370             sess = C.InferenceSession(session_options, self._model_path, True, self._read_config_from_model)
    371         else:
--> 372             sess = C.InferenceSession(session_options, self._model_bytes, False, self._read_config_from_model)
    373 
    374         if disabled_optimizers is None:

--KeyboardInterrupt--

KeyboardInterrupt: Interrupted by user
Build failed


-1